In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd

In [2]:
base_url = 'https://finance.naver.com/item/sise_day.nhn?code=066570&page='

In [3]:
response = requests.get(f'{base_url}1')

In [4]:
type(response.text)

str

In [5]:
html = bs(response.text)

In [6]:
type(html)

bs4.BeautifulSoup

In [7]:
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20200806200958" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="JavaScript">
function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
}
</script>
<h4 class="tlline2"><strong><span class="red03">일별</span>시세</strong></h4>
<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>

In [8]:
body > table.type2

NameError: name 'body' is not defined

In [9]:
body > table.type2 > tbody > tr:nth-child(6) > td:nth-child(3)

SyntaxError: invalid syntax (<ipython-input-9-ad43b419dab3>, line 1)

In [10]:
trs = html.select('body > table.type2 > tr')

In [11]:
trs[2].select('td')[0].get_text(strip = True)

'2020.08.11'

In [12]:
trs[2].select('td')[1].get_text(strip = True)

'83,400'

In [13]:
('+' if trs[2].select('img')[0]['alt'] else '-') + (trs[2].select('td')[2].get_text(strip = True))

'+2,300'

In [14]:
trs[2].select('td')[3].get_text(strip = True)

'86,300'

In [15]:
trs[2].select('td')[4].get_text(strip = True)

'86,300'

In [16]:
trs[2].select('td')[5].get_text(strip = True)

'83,100'

In [17]:
trs[2].select('td')[6].get_text(strip = True)

'1,207,107'

In [18]:
def parse_tr(tr):
    info = []
    
    date = tr.select('td')[0].get_text(strip = True)
    closing_price = tr.select('td')[1].get_text(strip = True)
    price = tr.select('td')[3].get_text(strip = True)
    high_price = tr.select('td')[4].get_text(strip = True)
    low_price = tr.select('td')[5].get_text(strip = True)
    total_price = tr.select('td')[6].get_text(strip = True)
    
    info.append(date)
    info.append(closing_price)
    info.append(price)
    info.append(high_price)
    info.append(low_price)
    info.append(total_price)
    
    return info

In [27]:
parse_tr(trs[2])

['2020.08.11', '83,400', '86,300', '86,300', '83,100', '1,207,107']

In [20]:
def stock_info(trs):
    stock_days = []

    
    for i in range(2, 15):
        if i in [7, 8, 9]:
            pass
        else:
            info = parse_tr(trs[i])
            stock_days.append(info)
        
    return stock_days

In [21]:
trs[15]

<tr>
<td colspan="7" height="8"></td>
</tr>

In [22]:
stock_info(trs)

[['2020.08.11', '83,400', '86,300', '86,300', '83,100', '1,207,107'],
 ['2020.08.10', '85,700', '77,100', '86,200', '76,500', '3,706,582'],
 ['2020.08.07', '77,200', '77,800', '78,300', '76,300', '759,800'],
 ['2020.08.06', '77,200', '78,300', '78,900', '76,900', '899,941'],
 ['2020.08.05', '78,300', '78,900', '78,900', '76,500', '1,197,933'],
 ['2020.08.04', '77,900', '74,900', '78,100', '74,000', '1,738,014'],
 ['2020.08.03', '74,000', '71,300', '74,500', '71,000', '1,183,563'],
 ['2020.07.31', '70,700', '72,200', '72,500', '70,600', '851,076'],
 ['2020.07.30', '72,100', '72,700', '73,500', '71,800', '735,701'],
 ['2020.07.29', '72,000', '72,500', '73,400', '71,700', '804,210']]

In [23]:
base_url

'https://finance.naver.com/item/sise_day.nhn?code=066570&page='

In [24]:
all_stock = []

def stock_naver_days(pnum):
    url = f'{base_url}{pnum}'
    response = requests.get(url)
    
    if response.status_code == 200:
        html = bs(response.text)
        trs = html.select('body > table.type2 > tr')
        return trs
    else:
        return False

In [25]:
stock_naver_days(3)

[<tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>,
 <tr>
 <td colspan="7" height="8"></td>
 </tr>,
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2020.07.14</span></td>
 <td class="num"><span class="tah p11">70,800</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				2,300
 				</span>
 </td>
 <td class="num"><span class="tah p11">68,700</span></td>
 <td class="num"><span class="tah p11">71,200</span></td>
 <td class="num"><span class="tah p11">68,600</span></td>
 <td class="num"><span class="tah p11">1,780,194</span></td>
 </tr>,
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2020.07.13</span></td>
 <td class="num"><span class="tah p11">68,500</span></td>
 <td cla

In [40]:
all_naver_stock = []

def stock_all_days_LG(pnum):
    trs = stock_naver_days(pnum)
    if pnum <= 453 :
        all_stock_LG = stock_info(trs)
        all_naver_stock.extend(all_stock_LG)
        
        pnum = pnum + 1
        
        return stock_all_days_LG(pnum)
    else :
        return all_naver_stock

In [41]:
stock_all_days_LG(1)

[['2020.08.11', '83,400', '86,300', '86,300', '83,100', '1,226,603'],
 ['2020.08.10', '85,700', '77,100', '86,200', '76,500', '3,706,582'],
 ['2020.08.07', '77,200', '77,800', '78,300', '76,300', '759,800'],
 ['2020.08.06', '77,200', '78,300', '78,900', '76,900', '899,941'],
 ['2020.08.05', '78,300', '78,900', '78,900', '76,500', '1,197,933'],
 ['2020.08.04', '77,900', '74,900', '78,100', '74,000', '1,738,014'],
 ['2020.08.03', '74,000', '71,300', '74,500', '71,000', '1,183,563'],
 ['2020.07.31', '70,700', '72,200', '72,500', '70,600', '851,076'],
 ['2020.07.30', '72,100', '72,700', '73,500', '71,800', '735,701'],
 ['2020.07.29', '72,000', '72,500', '73,400', '71,700', '804,210'],
 ['2020.07.28', '72,500', '74,000', '74,200', '72,100', '975,596'],
 ['2020.07.27', '73,600', '73,700', '75,300', '73,300', '568,984'],
 ['2020.07.24', '74,100', '73,400', '75,200', '73,000', '698,830'],
 ['2020.07.23', '74,100', '74,800', '74,800', '73,100', '759,138'],
 ['2020.07.22', '75,000', '75,000', '7

In [52]:
df = pd.DataFrame(all_naver_stock)
df.h()

,0,1,2,3,4,5
0,2020.08.11,"83,400","86,300","86,300","83,100","1,226,603"
1,2020.08.10,"85,700","77,100","86,200","76,500","3,706,582"
2,2020.08.07,"77,200","77,800","78,300","76,300","759,800"
3,2020.08.06,"77,200","78,300","78,900","76,900","899,941"
4,2020.08.05,"78,300","78,900","78,900","76,500","1,197,933"


In [53]:
header = ['일자', '종가', '시가', '고가', '저가', '거래량']

In [54]:
df.columns = header

In [84]:
df.drop(4529, inplace = True)

In [85]:
(df[['종가','시가','고가','저가','거래량']] == '').sum()

종가     0
시가     0
고가     0
저가     0
거래량    0
dtype: int64

In [86]:
df['일자'] = pd.to_datetime(df['일자'], format = '%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4529 entries, 0 to 4528
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      4529 non-null   datetime64[ns]
 1   종가      4529 non-null   object        
 2   시가      4529 non-null   object        
 3   고가      4529 non-null   object        
 4   저가      4529 non-null   object        
 5   거래량     4529 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 247.7+ KB


In [96]:
df[['종가','시가','고가','저가','거래량']] = df[['종가','시가','고가','저가','거래량']].apply(lambda x: x.str.replace(',' , '')).astype(int)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4529 entries, 0 to 4528
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      4529 non-null   datetime64[ns]
 1   종가      4529 non-null   int64         
 2   시가      4529 non-null   int64         
 3   고가      4529 non-null   int64         
 4   저가      4529 non-null   int64         
 5   거래량     4529 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 247.7 KB


In [98]:
df['전일비'] = (df['종가']) - (df['종가'].shift(1))
df.head(10)

,일자,종가,시가,고가,저가,거래량,전일비
0,2020-08-11,83400,86300,86300,83100,1226603,NaN
1,2020-08-10,85700,77100,86200,76500,3706582,2300.0
2,2020-08-07,77200,77800,78300,76300,759800,-8500.0
3,2020-08-06,77200,78300,78900,76900,899941,0.0
4,2020-08-05,78300,78900,78900,76500,1197933,1100.0
5,2020-08-04,77900,74900,78100,74000,1738014,-400.0
6,2020-08-03,74000,71300,74500,71000,1183563,-3900.0
7,2020-07-31,70700,72200,72500,70600,851076,-3300.0
8,2020-07-30,72100,72700,73500,71800,735701,1400.0
9,2020-07-29,72000,72500,73400,71700,804210,-100.0


In [102]:
df['전일비'] = df['전일비'].fillna(0).astype(int)
df.head()

,일자,종가,시가,고가,저가,거래량,전일비
0,2020-08-11,83400,86300,86300,83100,1226603,0
1,2020-08-10,85700,77100,86200,76500,3706582,2300
2,2020-08-07,77200,77800,78300,76300,759800,-8500
3,2020-08-06,77200,78300,78900,76900,899941,0
4,2020-08-05,78300,78900,78900,76500,1197933,1100


In [103]:
df.to_csv('Naver_finance_LG_electronic.csv', index = False)

In [104]:
pd.read_csv('Naver_finance_LG_electronic.csv')

,일자,종가,시가,고가,저가,거래량,전일비
0,2020-08-11,83400,86300,86300,83100,1226603,0
1,2020-08-10,85700,77100,86200,76500,3706582,2300
2,2020-08-07,77200,77800,78300,76300,759800,-8500
3,2020-08-06,77200,78300,78900,76900,899941,0
4,2020-08-05,78300,78900,78900,76500,1197933,1100
...,...,...,...,...,...,...,...
4524,2002-04-26,54800,51800,56000,50300,4176615,2600
4525,2002-04-25,52400,55000,56400,51800,3913730,-2400
4526,2002-04-24,57000,59500,60800,54300,4659371,4600
4527,2002-04-23,58800,62100,62800,58500,5384435,1800
